# ULKB Initialization

In [1]:
from itertools import islice
from ulkb import *   # import ULKB Logic namespace
settings.serializer.ulkb.ensure_ascii = False

# FOL reasoning

Below we directly encoded in ULKB the `PUZ001+1.p` from https://www.tptp.org/ website.

In [2]:
txt = ['Someone who lives in Dreadbury killed Agatha.',
       'Agatha, the butler, and Charles live in Dreadbury, and are the only people who live therein.',
       'A killer always hates his victim, and is never richer than his victim',
       'Charles hates no one that Agatha hates.',
       'Agatha hates everyone except the butler',
       'The butler hates everyone not richer than Agatha',
       'The butler hates everyone Agatha hates',
       'No one hates everyone',
       'Agatha is not the butler']

a             = TypeVariable('a') # a : *
ab            = FunctionType(a, BoolType()) # a -> bool : *
aab           = FunctionType(a, a, BoolType()) # a -> a -> bool : *
aaab           = FunctionType(a, a, a, BoolType())

lives         = new_constant("lives", ab)
killed        = new_constant("killed", aab)
hates         = new_constant("hates", aab)
richer        = new_constant("richer", aab)

agatha        = new_constant('agatha', a)
butler        = new_constant('butler', a) 
charles       = new_constant('charles', a)

x, y, w, z = Variables('x', 'y', 'w', 'z', a)

ax0  = Exists(x, And(lives(x), killed(x,agatha)))
ax1  = Forall(x, Implies(lives(x), Or(equal(x,agatha),equal(x,butler),equal(x,charles))))
ax21 = Forall(x, Forall(y, Implies(killed(x,y),hates(x,y))))
ax22 = Forall(x, y, Implies(killed(x,y), Not(richer(x,y))))
ax3  = Forall(x, Implies(hates(agatha,x), Not(hates(charles,x))))
ax4  = Forall(x, Implies(Not(equal(x,butler)), hates(agatha,x)))
ax5  = Forall(x, Implies(Not(richer(x, agatha)), hates(butler,x)))
ax6  = Forall(x, Implies(hates(agatha,x), hates(butler,x)))
# ax7  = Not(Exists(x, Forall(y, Implies(lives(y),hates(x,y)))))
ax7  = Not(Exists(x, Forall(y, hates(x,y))))
ax8  = Not(equal(agatha,butler))

In [3]:
for ax in [ax0,ax1,ax21,ax22,ax3,ax4,ax5,ax6,ax7,ax8]: 
    new_axiom(ax)

In [4]:
show_axioms()
RuleZ3(killed(agatha, agatha), timeout = 60)

26	axiom ∃ x, lives x ∧ killed x agatha
27	axiom ∀ x, lives x → x = agatha ∨ x = butler ∨ x = charles
28	axiom ∀ x y, killed x y → hates x y
29	axiom ∀ x y, killed x y → ¬richer x y
30	axiom ∀ x, hates agatha x → ¬hates charles x
31	axiom ∀ x, x ≠ butler → hates agatha x
32	axiom ∀ x, ¬richer x agatha → hates butler x
33	axiom ∀ x, hates agatha x → hates butler x
34	axiom ¬(∃ x, ∀ y, hates x y)
35	axiom agatha ≠ butler


⊢ killed agatha agatha

Whatelse can we derive from these axioms?

In [5]:
sentences = [
    Equal(butler, charles), Not(Equal(butler, charles)), 
    Equal(butler, agatha), Not(Equal(butler, agatha)), 
    Exists(x, y, z, And(lives(x), lives(y), lives(z), Not(Equal(x,y)), Not(Equal(x,z)), Not(Equal(y,z)))),
    Exists(x, y, And(lives(x), lives(y), Not(Equal(x,y)))),
    Exists(x, y, z, And(Not(Equal(x,y)), Not(Equal(x,z)), Not(Equal(y,z)))),
    Exists(x, y, z, w, And(Not(Equal(x,y)), Not(Equal(x,z)), Not(Equal(x,w)), 
                              Not(Equal(y,z)), Not(Equal(y,w)), 
                              Not(Equal(z,w))))]

for frm in sentences:
    try: 
        print(RuleZ3(frm))
    except:
        print(f"F {frm}")

F butler = charles : 𝔹
⊢ butler ≠ charles
F butler = agatha : 𝔹
⊢ butler ≠ agatha
F (∃ x y z, lives x ∧ lives y ∧ lives z ∧ x ≠ y ∧ x ≠ z ∧ y ≠ z) : 𝔹
F (∃ x y, lives x ∧ lives y ∧ x ≠ y) : 𝔹
⊢ ∃ x y z, x ≠ y ∧ x ≠ z ∧ y ≠ z
F (∃ x y z w, x ≠ y ∧ x ≠ z ∧ x ≠ w ∧ y ≠ z ∧ y ≠ w ∧ z ≠ w) : 𝔹


# MRS Logic

Consider the semantics of:

- Agatha is the only person who lives in Dreadbury.

or, we can't distinguish the two cases:

- Children who sing are happy.
- Children who are happy sing.

In [6]:
# ! killall ukb_wsd
%set_env ACE_GRAMMAR erg.dat
import mrs_logic
    
ctx = mrs_logic.get_current_context()
ctx.options['to_ulkb']['universe_type'] = a   # set the type of the (FOL) universe to "a"
ctx.options['to_ulkb']['drop_uvars'] = True   # delete "u" vars
ctx.options['to_ulkb']['drop_ivars'] = True   # delete "i" vars
ctx.options.to_ulkb.mk_q_exists = ['_the_q']

env: ACE_GRAMMAR=erg.dat


## Axioms

In [7]:
reset()

x, y, e1, e2 = Variables('x', 'y', 'e1', 'e2', a)

Agatha    = Constant('Agatha', a)
Charles    = Constant('Charles', a)
Dreadbury = Constant('Dreadbury', a)
person    = Constant('person', ab)
_kill_v_1 = Constant('_kill_v_1', aaab)
_in_p_dir = Constant('_in_p_dir', aaab)
_live_v_1 = Constant('_live_v_1', aab)
_rich_a_in = Constant('_rich_a_in', aab)
more_comp = Constant('more_comp', aaab)
_butler_n_1 = Constant('_butler_n_1', ab)
_hate_v_1 = Constant('_hate_v_1', aaab)
_be_v_id = Constant('_be_v_id', aaab)

new_axiom(Forall(x, y, Iff(Exists(e1, _be_v_id(e1,x,y)), equal(x,y))))

new_axiom(person(Agatha))
new_axiom(person(Charles))
new_axiom(Forall(x, Implies(_butler_n_1(x), person(x))))

show_axioms()

19	axiom ∀ x y, (∃ e1, _be_v_id e1 x y) ↔ x = y
20	axiom person Agatha
21	axiom person Charles
22	axiom ∀ x, _butler_n_1 x → person x


## Sentence 0

In [8]:
new_axiom(equal(Agatha, agatha))
new_axiom(Forall(x, Iff(Exists(e1, e2, y, And(person(x), _live_v_1(e1, x), _in_p_dir(e2, e1, y), Equal(Dreadbury,y))), lives(x))))
new_axiom(Forall(x, y, Iff(Exists(e1, _kill_v_1(e1, x, y)), killed(x,y))))
show_axioms()

sentence = txt[0]
solution = next(mrs_logic.solve(sentence)) # take the first solution 
form = solution.to_ulkb()
conjecture = Implies(form, ax0)
print(sentence)
try: 
    res = RuleZ3(conjecture, timeout = 30000)
    print(res)
except:
    print(conjecture)
    print(">>> FAILED")

19	axiom ∀ x y, (∃ e1, _be_v_id e1 x y) ↔ x = y
20	axiom person Agatha
21	axiom person Charles
22	axiom ∀ x, _butler_n_1 x → person x
23	axiom Agatha = agatha
24	axiom ∀ x, (∃ e1 e2 y, person x ∧ _live_v_1 e1 x ∧ _in_p_dir e2 e1 y ∧ Dreadbury = y) ↔ lives x
25	axiom ∀ x y, (∃ e1, _kill_v_1 e1 x y) ↔ killed x y
Someone who lives in Dreadbury killed Agatha.
⊢ (∃ x10, Dreadbury = x10 ∧ (∃ x3, (∃ e8 e9, person x3 ∧ _live_v_1 e8 x3 ∧ _in_p_dir e9 e8 x10) ∧ (∃ x16, Agatha = x16 ∧ (∃ e2, _kill_v_1 e2 x3 x16)))) → (∃ x, lives x ∧ killed x agatha)


## Sentence 1

In [9]:
x1, x2, x3, w, z, e = Variables('x1', 'x2', 'x3', 'w', 'z', 'e', a)

_people_n_of  = Constant('_people_n_of', FunctionType(a, bool))
implicit_conj = Constant('implicit_conj', FunctionType(a, a, a, bool))
_and_c        = Constant('_and_c', FunctionType(a, a, a, bool))
_only_a_1     = Constant('_only_a_1', FunctionType(a, a, bool))

def pax1(p1,p2):
    return Forall(x, y, z, Exists(x1, x2, e, 
                                  Iff(And(implicit_conj(x1, x, x2), _and_c(x2, y, z), p1(e, x1)),
                                      Exists(w, Implies(p2(w), Or(Equal(w,x), Equal(w,y), Equal(w,z)))))))

def pax1(p1,p2):
    return Forall(x, y, z, Exists(x1, x2, e, 
                                  Iff(And(implicit_conj(x1, x, x2), _and_c(x2, y, z), p1(e, x1)),
                                      And(p2(x), p2(y), p2(z)))))

def pax2(p1,p2):
    return Forall(x, y, z, Exists(x1, x2, e, 
                                  Iff(And(implicit_conj(x1, x, x2), _and_c(x2, y, z), p1(x1)),
                                      And(p2(x), p2(y), p2(z)))))

# new_axiom(pax1(_live_v_1,lives))
# new_axiom(pax2(_people_n_of,person))
show_axioms()

sentence = txt[1]
solution = next(mrs_logic.solve(sentence)) # take the first solution 
form = solution.to_ulkb()
conjecture = Implies(form, ax1)
print(sentence)
try: 
    res = RuleZ3(conjecture, timeout = 30000)
    print(res)
except:
    print(conjecture)
    print(">>> FAILED")   

19	axiom ∀ x y, (∃ e1, _be_v_id e1 x y) ↔ x = y
20	axiom person Agatha
21	axiom person Charles
22	axiom ∀ x, _butler_n_1 x → person x
23	axiom Agatha = agatha
24	axiom ∀ x, (∃ e1 e2 y, person x ∧ _live_v_1 e1 x ∧ _in_p_dir e2 e1 y ∧ Dreadbury = y) ↔ lives x
25	axiom ∀ x y, (∃ e1, _kill_v_1 e1 x y) ↔ killed x y
Agatha, the butler, and Charles live in Dreadbury, and are the only people who live therein.
(∃ x32, Dreadbury = x32 ∧ (∃ x14, (∃ x24, Charles = x24 ∧ (∃ x19, _butler_n_1 x19 ∧ _and_c x14 x19 x24)) ∧ (∃ x39, (∃ e44 e46 e47, _only_a_1 e44 x39 ∧ _people_n_of x39 ∧ _live_v_1 e46 x39 ∧ _therein_p_dir e47 e46) ∧ (∃ x3, (∃ x8, Agatha = x8 ∧ implicit_conj x3 x8 x14) ∧ (∃ e30 e31 e2 e38, _live_v_1 e30 x3 ∧ _in_p_dir e31 e30 x32 ∧ _and_c e2 e30 e38 ∧ _be_v_id e38 x3 x39))))) → (∀ x, lives x → x = agatha ∨ x = butler ∨ x = charles) : 𝔹
>>> FAILED


## Sentence 2

In [10]:
show_axioms()

sentence = txt[2]
solution = next(mrs_logic.solve(sentence)) # take the first solution 
form = solution.to_ulkb()
conjecture = Implies(form, And(ax21, ax22))
print(sentence)
try: 
    res = RuleZ3(conjecture, timeout = 30000)
    print(res)
except:
    print(conjecture)
    print(">>> FAILED")

19	axiom ∀ x y, (∃ e1, _be_v_id e1 x y) ↔ x = y
20	axiom person Agatha
21	axiom person Charles
22	axiom ∀ x, _butler_n_1 x → person x
23	axiom Agatha = agatha
24	axiom ∀ x, (∃ e1 e2 y, person x ∧ _live_v_1 e1 x ∧ _in_p_dir e2 e1 y ∧ Dreadbury = y) ↔ lives x
25	axiom ∀ x y, (∃ e1, _kill_v_1 e1 x y) ↔ killed x y
A killer always hates his victim, and is never richer than his victim
(∃ e2 e22 e9 x10, (∃ x16, pron x16 ∧ (∃ e15, poss e15 x10 x16 ∧ _victim_n_of x10)) ∧ (∃ x34, pron x34 ∧ (∃ x3, _killer_n_1 x3 ∧ (∃ e22, _always_a_1 e9 ∧ _hate_v_1 e9 x3 x10 ∧ _and_c e2 e9 e22 ∧ _never_a_1 e22 (∃ x28, (∃ e9 e9 e2 e33, poss e33 x28 x34 ∧ _victim_n_of x28) ∧ (∃ e25 e27, _rich_a_in e25 x3 ∧ more_comp e27 e25 x28)))))) → (∀ x y, killed x y → hates x y) ∧ (∀ x y, killed x y → ¬richer x y) : 𝔹
>>> FAILED


## Sentence 3

In [11]:
new_axiom(equal(Charles, charles))
new_axiom(Forall(x, y, Iff(Exists(e1, _hate_v_1(e1,x,y)), hates(x,y))))
new_axiom(Forall(x, y, Implies(hates(x,y), And(person(x),person(y)))))
show_axioms()

sentence = txt[3]
solutions = list(islice(mrs_logic.solve(sentence), 10)) 
forms = [solution.to_ulkb() for solution in solutions]
form = forms[2] # best solution 
conjecture = Implies(form, ax3)
print(sentence)
try: 
    res = RuleZ3(conjecture, timeout = 30000)
    print(res)
except:
    print(conjecture)
    print(">>> FAILED")

19	axiom ∀ x y, (∃ e1, _be_v_id e1 x y) ↔ x = y
20	axiom person Agatha
21	axiom person Charles
22	axiom ∀ x, _butler_n_1 x → person x
23	axiom Agatha = agatha
24	axiom ∀ x, (∃ e1 e2 y, person x ∧ _live_v_1 e1 x ∧ _in_p_dir e2 e1 y ∧ Dreadbury = y) ↔ lives x
25	axiom ∀ x y, (∃ e1, _kill_v_1 e1 x y) ↔ killed x y
26	axiom Charles = charles
27	axiom ∀ x y, (∃ e1, _hate_v_1 e1 x y) ↔ hates x y
28	axiom ∀ x y, hates x y → person x ∧ person y
Charles hates no one that Agatha hates.
⊢ (∃ x15, Agatha = x15 ∧ (∃ x3, Charles = x3 ∧ (∀ x9, (∃ e20, person x9 ∧ _hate_v_1 e20 x15 x9) → ¬(∃ e2, _hate_v_1 e2 x3 x9)))) → (∀ x, hates agatha x → ¬hates charles x)


## Sentence 4

In [12]:
show_axioms()

sentence = txt[4]
solution = next(mrs_logic.solve(sentence)) # take the first solution 
form = solution.to_ulkb()
conjecture = Implies(form, ax4)
print(sentence)
try: 
    res = RuleZ3(conjecture, timeout = 30000)
    print(res)
except:
    print(conjecture)
    print(">>> FAILED")

19	axiom ∀ x y, (∃ e1, _be_v_id e1 x y) ↔ x = y
20	axiom person Agatha
21	axiom person Charles
22	axiom ∀ x, _butler_n_1 x → person x
23	axiom Agatha = agatha
24	axiom ∀ x, (∃ e1 e2 y, person x ∧ _live_v_1 e1 x ∧ _in_p_dir e2 e1 y ∧ Dreadbury = y) ↔ lives x
25	axiom ∀ x y, (∃ e1, _kill_v_1 e1 x y) ↔ killed x y
26	axiom Charles = charles
27	axiom ∀ x y, (∃ e1, _hate_v_1 e1 x y) ↔ hates x y
28	axiom ∀ x y, hates x y → person x ∧ person y
Agatha hates everyone except the butler
(∀ x9, (∃ x15, _butler_n_1 x15 ∧ (∃ e14, person x9 ∧ _except_p e14 x9 x15)) → (∃ x3, Agatha = x3 ∧ (∃ e2, _hate_v_1 e2 x3 x9))) → (∀ x, x ≠ butler → hates agatha x) : 𝔹
>>> FAILED


## Sentence 5

In [13]:
new_axiom(Forall(x,y, Iff(Exists(e1, e2, And(_rich_a_in(e1, x), more_comp(e2, e1, y))), richer(x,y))))
new_axiom(Forall(x, y, Implies(Not(richer(x,y)), And(person(x),person(y)))))
new_axiom(Forall(x, Iff(_butler_n_1(x), equal(x,butler))))
show_axioms()

sentence = txt[5]
solutions = list(islice(mrs_logic.solve(sentence), 10)) 
forms = [solution.to_ulkb() for solution in solutions]
form = forms[2] # best solution 
conjecture = Implies(form, ax5)
print(sentence)
try: 
    res = RuleZ3(conjecture, timeout = 30000)
    print(res)
except:
    print(conjecture)
    print(">>> FAILED")

19	axiom ∀ x y, (∃ e1, _be_v_id e1 x y) ↔ x = y
20	axiom person Agatha
21	axiom person Charles
22	axiom ∀ x, _butler_n_1 x → person x
23	axiom Agatha = agatha
24	axiom ∀ x, (∃ e1 e2 y, person x ∧ _live_v_1 e1 x ∧ _in_p_dir e2 e1 y ∧ Dreadbury = y) ↔ lives x
25	axiom ∀ x y, (∃ e1, _kill_v_1 e1 x y) ↔ killed x y
26	axiom Charles = charles
27	axiom ∀ x y, (∃ e1, _hate_v_1 e1 x y) ↔ hates x y
28	axiom ∀ x y, hates x y → person x ∧ person y
29	axiom ∀ x y, (∃ e1 e2, _rich_a_in e1 x ∧ more_comp e2 e1 y) ↔ richer x y
30	axiom ∀ x y, ¬richer x y → person x ∧ person y
31	axiom ∀ x, _butler_n_1 x ↔ x = butler
The butler hates everyone not richer than Agatha
⊢ (∃ x19, Agatha = x19 ∧ (∀ x8, person x8 ∧ ¬(∃ e16 e18, _rich_a_in e16 x8 ∧ more_comp e18 e16 x19) → (∃ x3, _butler_n_1 x3 ∧ (∃ e2, _hate_v_1 e2 x3 x8)))) → (∀ x, ¬richer x agatha → hates butler x)


## Sentence 6

In [14]:
show_axioms()

sentence = txt[6]
solution = next(mrs_logic.solve(sentence)) # take the first solution 
form = solution.to_ulkb()
conjecture = Implies(form, ax6)
print(sentence)
try: 
    res = RuleZ3(conjecture, timeout = 30000)
    print(res)
except:
    print(conjecture)
    print(">>> FAILED")

19	axiom ∀ x y, (∃ e1, _be_v_id e1 x y) ↔ x = y
20	axiom person Agatha
21	axiom person Charles
22	axiom ∀ x, _butler_n_1 x → person x
23	axiom Agatha = agatha
24	axiom ∀ x, (∃ e1 e2 y, person x ∧ _live_v_1 e1 x ∧ _in_p_dir e2 e1 y ∧ Dreadbury = y) ↔ lives x
25	axiom ∀ x y, (∃ e1, _kill_v_1 e1 x y) ↔ killed x y
26	axiom Charles = charles
27	axiom ∀ x y, (∃ e1, _hate_v_1 e1 x y) ↔ hates x y
28	axiom ∀ x y, hates x y → person x ∧ person y
29	axiom ∀ x y, (∃ e1 e2, _rich_a_in e1 x ∧ more_comp e2 e1 y) ↔ richer x y
30	axiom ∀ x y, ¬richer x y → person x ∧ person y
31	axiom ∀ x, _butler_n_1 x ↔ x = butler
The butler hates everyone Agatha hates
⊢ (∀ x8, (∃ x14, Agatha = x14 ∧ (∃ e19, person x8 ∧ _hate_v_1 e19 x14 x8)) → (∃ x3, _butler_n_1 x3 ∧ (∃ e2, _hate_v_1 e2 x3 x8))) → (∀ x, hates agatha x → hates butler x)


## Sentence 7

In [15]:
show_axioms()

sentence = txt[7]
solutions = list(islice(mrs_logic.solve(sentence), 10)) 
forms = [solution.to_ulkb() for solution in solutions]
form = forms[1]
conjecture = Implies(form, ax7)
print(sentence)
try: 
    res = RuleZ3(conjecture, timeout = 120000)
    print(res)
except:
    print(conjecture)
    print(">>> FAILED")

19	axiom ∀ x y, (∃ e1, _be_v_id e1 x y) ↔ x = y
20	axiom person Agatha
21	axiom person Charles
22	axiom ∀ x, _butler_n_1 x → person x
23	axiom Agatha = agatha
24	axiom ∀ x, (∃ e1 e2 y, person x ∧ _live_v_1 e1 x ∧ _in_p_dir e2 e1 y ∧ Dreadbury = y) ↔ lives x
25	axiom ∀ x y, (∃ e1, _kill_v_1 e1 x y) ↔ killed x y
26	axiom Charles = charles
27	axiom ∀ x y, (∃ e1, _hate_v_1 e1 x y) ↔ hates x y
28	axiom ∀ x y, hates x y → person x ∧ person y
29	axiom ∀ x y, (∃ e1 e2, _rich_a_in e1 x ∧ more_comp e2 e1 y) ↔ richer x y
30	axiom ∀ x y, ¬richer x y → person x ∧ person y
31	axiom ∀ x, _butler_n_1 x ↔ x = butler
No one hates everyone
⊢ (∀ x3, person x3 → ¬(∀ x8, person x8 → (∃ e2, _hate_v_1 e2 x3 x8))) → ¬(∃ x, ∀ y, hates x y)


## Sentence 8

In [16]:
show_axioms()

sentence = txt[8]
solution = next(mrs_logic.solve(sentence)) # take the first solution 
form = solution.to_ulkb()
conjecture = Implies(form, ax8)
print(sentence)
try: 
    res = RuleZ3(conjecture, timeout = 30000)
    print(res)
except:
    print(conjecture)
    print(">>> FAILED")

19	axiom ∀ x y, (∃ e1, _be_v_id e1 x y) ↔ x = y
20	axiom person Agatha
21	axiom person Charles
22	axiom ∀ x, _butler_n_1 x → person x
23	axiom Agatha = agatha
24	axiom ∀ x, (∃ e1 e2 y, person x ∧ _live_v_1 e1 x ∧ _in_p_dir e2 e1 y ∧ Dreadbury = y) ↔ lives x
25	axiom ∀ x y, (∃ e1, _kill_v_1 e1 x y) ↔ killed x y
26	axiom Charles = charles
27	axiom ∀ x y, (∃ e1, _hate_v_1 e1 x y) ↔ hates x y
28	axiom ∀ x y, hates x y → person x ∧ person y
29	axiom ∀ x y, (∃ e1 e2, _rich_a_in e1 x ∧ more_comp e2 e1 y) ↔ richer x y
30	axiom ∀ x y, ¬richer x y → person x ∧ person y
31	axiom ∀ x, _butler_n_1 x ↔ x = butler
Agatha is not the butler
⊢ ¬(∃ x10, _butler_n_1 x10 ∧ (∃ x3, Agatha = x3 ∧ (∃ e2, _be_v_id e2 x3 x10))) → agatha ≠ butler


In [ ]:
RuleZ3( Falsity(), timeout = 240000 )

## Draft 

1. we can't prove sentences 1, 2, 4, 7

2. Can you think in a sentence where udef_q need to be interpreted as

(\exists x, RSTR and BODY) and not (\forall x, RSTR -> BODY)?

Several examples:
1. nonhead noun in N-N compounds: The dog owner appeared. [quantifier for "dog" variable]
2. Prepositional phrases with a determinerless nominal: "He read the book from beginning to end."
3. coordination with identity: "My friend and neighbor often helps me."


In [17]:
sol = next(mrs_logic.solve(txt[1]))
form = sol.to_ulkb()
print(sol.mrs)
print(form)
form.constants

[ TOP: h0
  INDEX: e2
  RELS: < [ udef_q LBL: h4 ARG0: x3 RSTR: h5 BODY: h6 ]
          [ proper_q LBL: h7 ARG0: x8 RSTR: h9 BODY: h10 ]
          [ named LBL: h11 ARG0: x8 CARG: "Agatha" ]
          [ udef_q LBL: h13 ARG0: x14 RSTR: h15 BODY: h16 ]
          [ implicit_conj LBL: h17 ARG0: x3 ARG1: x8 ARG2: x14 ]
          [ _the_q LBL: h18 ARG0: x19 RSTR: h20 BODY: h21 ]
          [ _butler_n_1 LBL: h22 ARG0: x19 ]
          [ _and_c LBL: h23 ARG0: x14 ARG1: x19 ARG2: x24 ]
          [ proper_q LBL: h25 ARG0: x24 RSTR: h26 BODY: h27 ]
          [ named LBL: h28 ARG0: x24 CARG: "Charles" ]
          [ _live_v_1 LBL: h1 ARG0: e30 ARG1: x3 ]
          [ _in_p_dir LBL: h1 ARG0: e31 ARG1: e30 ARG2: x32 ]
          [ proper_q LBL: h33 ARG0: x32 RSTR: h34 BODY: h35 ]
          [ named LBL: h36 ARG0: x32 CARG: "Dreadbury" ]
          [ _and_c LBL: h1 ARG0: e2 ARG1: e30 ARG2: e38 ]
          [ _be_v_id LBL: h1 ARG0: e38 ARG1: x3 ARG2: x39 ]
          [ _the_q LBL: h40 ARG0: x39 RSTR: h41 BODY:

frozenset({Agatha : a,
           Charles : a,
           Dreadbury : a,
           _and_c : a → a → a → 𝔹,
           _be_v_id : a → a → a → 𝔹,
           _butler_n_1 : a → 𝔹,
           _in_p_dir : a → a → a → 𝔹,
           _live_v_1 : a → a → 𝔹,
           _only_a_1 : a → a → 𝔹,
           _people_n_of : a → 𝔹,
           _therein_p_dir : a → a → 𝔹,
           and : 𝔹 → 𝔹 → 𝔹,
           equal : a → a → 𝔹,
           exists : (a → 𝔹) → 𝔹,
           implicit_conj : a → a → a → 𝔹})